In [1]:
#we just do it once
#!pip3 install xgboost

In [1]:
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from pathlib import Path
import functools

In [3]:
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
#loading the data quant

path1=Path("/home/znazari/machine_learning_ppmi")

quant_rna1=pd.read_csv(path1/"ordered_bl_matrix_ir2_quant.txt", header=None,index_col=[0] )
quant_rna=quant_rna1.rename(columns=quant_rna1.iloc[0]).drop(quant_rna1.index[0]).reset_index(drop=True)

In [5]:
#loading the quant diagnosis: prediction

quant_predict1=pd.read_csv(path1/"the_diagnosis_quant_ir2.txt", header=None ,index_col=[0])
quant_predict=quant_predict1.rename(columns=quant_predict1.iloc[0]).drop(quant_predict1.index[0]).reset_index(drop=True)

In [6]:
#droping geneid because we need to do some manipulation on integer data
quant_rna_=quant_rna.drop(columns='Geneid')

In [7]:
a_quant_rna=quant_rna_.add(1)

In [8]:
log_quant_rna=np.log2(a_quant_rna)

In [9]:
d_log_quant_rna=pd.DataFrame(log_quant_rna)

In [10]:
# plus one and log2 transformed data of rna seq quant ir2
d_log_quant_rna

,10874.0,12499.0,12593.0,13039.0,13424.0,14281.0,14331.0,14426.0,15761.0,16580.0,...,59614.0,60002.0,60015.0,60024.0,60036.0,60095.0,60171.0,65002.0,65003.0,85242.0
0,4.600412,7.489463,6.218715,6.243604,8.330796,7.310867,6.646383,6.195139,7.259404,5.334014,...,6.658483,6.602273,7.144974,7.208400,7.952066,6.542750,7.172868,4.983942,7.313019,10.089013
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,1.020043,0.000000,0.553211,0.000000,0.000000,0.000000,0.096820,0.000000,0.017498,...,0.000000,0.452357,0.433116,0.791722,0.000000,0.363370,1.841039,0.000000,0.396550,0.072359
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34564,5.261395,5.111244,5.189208,5.704139,4.849929,4.507471,4.864191,4.699502,4.739259,4.750912,...,4.699924,5.357042,5.334354,5.405237,4.895623,5.032502,5.127270,4.201963,4.871647,5.363718
34565,0.251088,0.000000,0.630849,0.000000,0.000000,0.496756,0.000000,0.000000,0.588092,0.000000,...,0.000000,0.310789,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.129031,0.000000
34566,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.905263,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
34567,0.655203,0.000000,0.000000,0.000000,0.000000,2.195281,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
#transposed matrix
t_quant=d_log_quant_rna.T
t_quant

,0,1,2,3,4,5,6,7,8,9,...,34559,34560,34561,34562,34563,34564,34565,34566,34567,34568
10874.0,4.600412,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,...,1.300381,0.812215,0.000000,0.047000,0.000000,5.261395,0.251088,0.0,0.655203,3.412031
12499.0,7.489463,0.0,0.0,1.020043,0.0,0.000000,0.000000,0.255966,0.0,0.17473,...,1.608748,0.293551,0.000000,0.210991,0.000000,5.111244,0.000000,0.0,0.000000,3.794052
12593.0,6.218715,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,...,1.929772,0.950583,0.369857,0.000000,0.000000,5.189208,0.630849,0.0,0.000000,4.322022
13039.0,6.243604,0.0,0.0,0.553211,0.0,0.100292,0.000000,0.000000,0.0,0.00000,...,1.156060,0.000000,0.184626,0.751670,0.349870,5.704139,0.000000,0.0,0.000000,3.226920
13424.0,8.330796,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,...,1.832035,0.832289,0.000000,0.000000,0.169653,4.849929,0.000000,0.0,0.000000,3.975502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60095.0,6.542750,0.0,0.0,0.363370,0.0,0.000000,0.000000,0.000000,0.0,0.00000,...,1.184991,1.155315,0.000000,0.000000,0.242353,5.032502,0.000000,0.0,0.000000,3.719096
60171.0,7.172868,0.0,0.0,1.841039,0.0,0.000000,0.000000,0.000000,0.0,0.00000,...,0.961086,0.416584,0.000000,0.000000,0.000000,5.127270,0.000000,0.0,0.000000,3.089874
65002.0,4.983942,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,...,0.798477,0.000000,0.000000,0.000000,0.530963,4.201963,0.000000,0.0,0.000000,2.861451
65003.0,7.313019,0.0,0.0,0.396550,0.0,0.000000,0.210292,0.000000,0.0,0.00000,...,0.958631,0.336278,0.151351,0.000000,0.000000,4.871647,0.129031,0.0,0.000000,3.546981


In [12]:
searchfor = ['Healthy Control', "Parkinson's Disease"]


In [19]:
#we just pick those that are either health control or parkinson'd diseases
quant_p=quant_predict[quant_predict['COHORT_DEFINITION'].astype(str).str.contains('|'.join(searchfor))]

In [20]:
quant_new_predict=quant_p['BL_PATNO']

In [21]:
r_quant_patients=[int(x) for x in quant_new_predict]

In [22]:
#here we select those rna-seq data that includes both health control and parkinson 
selected_quant_rna=d_log_quant_rna[r_quant_patients]

In [23]:
select_quant_rna=selected_quant_rna.T

In [24]:
quant_p['COHORT_DEFINITION'].replace(['Healthy Control', "Parkinson's Disease"],
                        [0, 1], inplace=True)

/tmp/ipykernel_24098/93247712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quant_p['COHORT_DEFINITION'].replace(['Healthy Control', "Parkinson's Disease"],


In [25]:
#here we split the data quant
X_trainq, X_testq, y_trainq, y_testq = train_test_split(select_quant_rna,quant_p['COHORT_DEFINITION'], test_size=.3)


In [26]:
#here we make the model
bst_q = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')


In [27]:
#here we fit the model
bst_q.fit(X_trainq, y_trainq)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=2, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [28]:
#here we chech the prediction
preds_q = bst_q.predict(X_testq)


In [29]:
#let us see how the quant normalized data is doing
#predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_testq, preds_q)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 66.81%
